In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.dataset.dataset import TextDataset
import torch 
from torch import nn

In [3]:
data = TextDataset(
    corpus_path = 'data/corpus.txt',
    json_path='data/tiny_stories',
    save_tokenizer_to = 'bpe',
    max_len = 256,
    vocab_size = 5000
)
print('created train data with size:', len(data))

0it [00:00, ?it/s]

created train data with size: 20869767


In [4]:
generator = torch.Generator().manual_seed(42)
datasets = torch.utils.data.random_split(data, [0.95, 0.05], generator=generator)
train_data = datasets[0]
test_data = datasets[1]
print(len(train_data))
print(len(test_data))

19826279
1043488


In [4]:
from torch.utils.data import DataLoader
from src.model.language_model import LanguageModel
from src.dataset.dataset import Collator
from src.utils.trainer import CosineAnnealingWithWarmupLR

collate_fn = Collator(pad_value=data.pad_id)
train_loader = DataLoader(train_data, batch_size=768, shuffle=True, num_workers=4, collate_fn=collate_fn)
test_loader = DataLoader(test_data, batch_size=768, shuffle=False, num_workers=4, collate_fn=collate_fn)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LanguageModel(
    embed_dim=1024,
    vocab_size=data.vocab_size,
    max_len=data.max_len,
    pad_idx=data.pad_id,
    num_layers=16,
    num_heads=32,
    dropout=0,
    feedforward_dim=2048
)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
# lr_scheduler = CosineAnnealingWithWarmupLR(optimizer=optimizer, warmup_steps=50, max_steps=200)

In [5]:
print(model)

LanguageModel(
  (transformer): TransformerDecoder(
    (embedding): Embedding(5000, 1024, padding_idx=0)
    (pos_encoder): PositionalEncoding()
    (decoder): ModuleList(
      (0-15): 16 x DecoderBlock(
        (Q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (K_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (V_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (masked_multihead_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
        )
        (lay_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (feedforward): Sequential(
          (0): Linear(in_features=1024, out_features=2048, bias=True)
          (1): ReLU()
          (2): Linear(in_features=2048, out_features=1024, bias=True)
        )
        (dropout): Dropout(p=0, inplace=False)
        (lay_norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)

In [7]:
from src.utils.wandb_logger import WandbLogger
from src.utils.trainer import train

wdb = WandbLogger(
    project_name="little-lama",
    config={"loh": "loh"}
)
train(
    model, optimizer, criterion, train_loader, 
    train_loader, 100, DEVICE, wdb, 
    log_output = False, grad_clipping=10
)

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


=== Epoch 1 ===


100%|██████████| 4/4 [00:00<00:00, 11.81it/s]


=== Epoch 2 ===


100%|██████████| 4/4 [00:00<00:00, 16.85it/s]


...

=== Epoch 48 ===


100%|██████████| 4/4 [00:00<00:00, 12.15it/s]


=== Epoch 49 ===


100%|██████████| 4/4 [00:00<00:00, 14.02it/s]


KeyboardInterrupt: 

итоговый ран из wandb